In [ ]:
import os
import requests
import re
import pandas as pd
import numpy as np
from gen_python import generate_access_token
from PyPDF2 import PdfFileReader, PdfReader
from io import BytesIO

os.chdir('C:/Users/LENOVO/OneDrive/side_projects/Python_proyectos/lectura_pdfs_sunat')
APP_ID = '6b01b87e-1806-4dc1-8bfb-00e8b9531179' # aplicacion utilizada para la prueba. NO va a funcionar si lo corre otro usuario
SCOPES = ['User.Read','Files.Read','Files.Read.All','Files.ReadWrite','Files.ReadWrite.All','Sites.Read.All','Sites.ReadWrite.All']

access_token = generate_access_token(APP_ID, scopes=SCOPES)
headers = {'Authorization': 'Bearer ' + access_token['access_token']}

### Aqui se hacen las consultas
ruta = '/SUNAT/archivos_pdf' # ruta utilizada para la prueba. NO va a funcionar si lo corre otro usuario
n_pdfs = requests.get('https://graph.microsoft.com/v1.0/me/drive/root:' + ruta, headers=headers).json().get('folder').get('childCount')
lista_json = requests.get('https://graph.microsoft.com/v1.0/me/drive/root:' + ruta + ':/children', headers=headers).json()

nan_value = float('NaN')

db = pd.DataFrame(columns = ['verga','row_n','ident','tip','archivo_pdf'])
dbdb = pd.DataFrame(columns = ['archivo','ruc','nombre','nro_cpe','tipo','importe_total','fecha_emision','fecha_cuota','importe_cuota','descripcion'])

for i in range(1,n_pdfs+1):
    id_ = lista_json['value'][i-1]['id']
    name = lista_json['value'][i-1]['name']

    link = 'https://graph.microsoft.com/v1.0/me/drive/items/' + id_ + '/content'
    response = requests.get(link, headers=headers)

    pdf = PdfFileReader(BytesIO(response.content))
    pdf_m = pd.DataFrame(pdf.pages[0].
                         extract_text().
                         upper().
                         replace('\xa0','').
                         split('\n')).replace([' ',''],'').rename(columns = {0:'verga'})
    # .dropna()
    pdf_m['row_n'] = np.arange(len(pdf_m))
    pdf_m['ident'] = 'AZUCAR'
    pdf_m['tip'] = 'SABOR'

    d = 0
    while pdf_m.iloc[d]['verga'] == '':
        d = d + 1

    indexes = pd.DataFrame(
        {
            'archivo':[np.nan],
            'ruc':[np.nan],
            'nombre':[np.nan],
            'nro_cpe':[np.nan],
            'tipo':[np.nan],
            'importe_total':[np.nan],
            'fecha_emision':[np.nan],
            'fecha_cuota':[np.nan],
            'importe_cuota':[np.nan],
            'descripcion':[np.nan]
        }
    )
    
    for i in range(len(pdf_m)):
        check = pdf_m.iloc[i,0]
        indexes.iat[0,0] = name
        if bool(re.search('RUC\s*:\s*',check)) == True and bool(re.search('20606739193',check)) == False:
            indexes.iat[0,1] = re.sub('RUC\s*:\s*','',pdf_m.iloc[i,0])
        elif re.search('IMPORTE TOTAL',check):
            indexes.iat[0,5] = re.sub('IMPORTE TOTAL\s*:\s*S/\s*','',pdf_m.iloc[i,0])
        elif re.search('FORMA DE PAGO',check):
            if re.search('CONTADO',pdf_m.iloc[i,0]):
                indexes.iat[0,4] = 'CONTADO'
            else:
                indexes.iat[0,4] = 'CREDITO'
            k = 0
            conco = ''
            while pdf_m.iat[i+2+k,0] != '':
                conco = conco + ' ' + re.sub('\d\.\d\d\sUNIDAD\s','',pdf_m.iloc[i+2+k,0])
                k += 1
            indexes.iat[0,9] = conco
        elif re.search('FECHA DE EMISI',check):
            indexes.iat[0,6] = re.sub('.+\s*:\s*','',pdf_m.iloc[i,0])
        elif pdf_m.iloc[i,1] == d:
            indexes.iat[0,2] = pdf_m.iloc[i,0]
        elif re.search('E\\d\\d\\d',check):
            indexes.iat[0,3] = pdf_m.iloc[i,0]
        elif re.search('TOTAL DE CUOTAS',check):
            indexes.iat[0,7] = re.search('1(\d{2}\/\d{2}\/\d{4}).',pdf_m.iloc[i+2,0]).group(1)
            indexes.iat[0,8] = re.sub('\s*Nº CUOTA.*','',re.sub('1\d{2}\/\d{2}\/\d{4}','',pdf_m.iloc[i+2,0]))
        else:
            pdf_m.iloc[i,3] = 'NIMI'

    dbdb = pd.concat([dbdb,indexes])

    
dbdb.to_excel('facturas_octubre1.xlsx')
    